In [1]:
%env CUDA_DEVICE_ORDER=1
%env CUDA_VISIBLE_DEVICES=0

from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

env: CUDA_DEVICE_ORDER=1
env: CUDA_VISIBLE_DEVICES=0


Using TensorFlow backend.
/Users/mich/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9592228510114148630
]


In [3]:
#df_train = pd.read_pickle("./data/df_train.pkl")
#df_test = pd.read_pickle("./data/df_test.pkl")

token_train = pd.read_pickle("./data/token_train.pkl")
token_test = pd.read_pickle("./data/token_test.pkl")

#vec_train = pd.read_pickle("./data/vec_train_glove.pkl")
#vec_test = pd.read_pickle("./data/vec_test_glove.pkl")

In [ ]:
token_train.head()

# Word2Vec Embedding 

In [4]:
googlenews_embedding = './data/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(googlenews_embedding, binary=True)

vocabulary = dict()
inverse_vocabulary = ['<unk>']

In [5]:
stops = set(stopwords.words('english'))
questions = ['tokenq1', 'tokenq2']

# Iterate over the questions of both training and test datasets
for dataset in [token_train, token_test]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions:

            question_to_number = []  # q2n -> question numbers representation
            for word in row[question]:

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    question_to_number.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    question_to_number.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, question_to_number)

/Users/mich/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [ ]:
token_train.head()

In [7]:
word2vec.save("word2vec")

In [16]:
x = word2vec.load('word2vec')

In [18]:
# Build the embedding matrix
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

# Prepare training and validation data

In [19]:
# split to train validation
validation_size = 40000
training_size = len(token_train) - validation_size

question_cols = token_train[['tokenq1', 'tokenq2']]
duplicate = token_train['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(question_cols, duplicate, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.tokenq1, 'right': X_train.tokenq2}
X_validation = {'left': X_validation.tokenq1, 'right': X_validation.tokenq2}
X_test = {'left': token_test.tokenq1, 'right': token_test.tokenq1}

In [20]:
# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

In [21]:
#max_seq_length = max(token_train.tokenq1.map(lambda x: len(x)).max(),
#                     token_train.tokenq2.map(lambda x: len(x)).max(),
#                     token_test.tokenq1.map(lambda x: len(x)).max(),
#                     token_test.tokenq2.map(lambda x: len(x)).max())
max_seq_length = 10

In [22]:
# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

In [23]:
X_train

{'left': array([[    0,     0,     0, ...,  2383,  5564, 33952],
        [    0,     0,   676, ...,  9070,   416,  8371],
        [    0,     0,     0, ..., 64859,     8,  2881],
        ..., 
        [ 2026,  1450,  1585, ...,   424,  4447,  5274],
        [    0,     0,     0, ...,   139,   342, 32493],
        [    2,     3,   180, ...,   576,  1954,  2503]], dtype=int32),
 'right': array([[    0,     0,     0, ..., 33952,  1585,   605],
        [ 2196,     8,   676, ...,    33,  1741,   439],
        [    0,     0,     1, ...,  2979,     8,  2881],
        ..., 
        [  872,  3284,  2526, ..., 11544,   164,   129],
        [    0,     0,     0, ...,   139,   342, 32493],
        [   23,    24,   315, ...,   865,  1954,  2503]], dtype=int32)}

In [24]:
# TEST
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

# Building the model

In [25]:
#n_hidden = 50
n_hidden = 1
gradient_clipping_norm = 1.25
#batch_size = 64
batch_size = 1
#n_epoch = 25
n_epoch = 10

In [26]:
# MaLSTM similarity function
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [27]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

In [28]:
# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

/Users/mich/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [29]:
# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

In [30]:
# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation), validation_split = 0.2)

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

Train on 283162 samples, validate on 40000 samples
Epoch 1/10
  9771/283162 [>.............................] - ETA: 59:44 - loss: 0.4326 - acc: 0.4112        

# Plotting results

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()